# Prepare DoS Hulk Attack Dataset

## Overview:

This notebook will focus on creating a DoS Hulk attack dataset based on a small sample of data collected by performing real DoS HTTP GET Flood attacks in a controlled environment.<br>
The dataset that this notebook creates closely represents real-world data and was used to train our SVM model.<br>  
It is worth noteing that the sample dataset we collected does not contain any missing values or any outliers due to the fact we tested each part of the collection process and verified that it is correct.<br>
In this notebook we have generated an attack dataset with 25,000 flows of the DoS HTTP GET Flood attack based on the samples we collected when running a DoS HTTP GET Flood attacks in various configurations using the well known DoS Hulk tool when the victim web server was online.<br> 
The victim web server was a regular Flask web server.

## Imports & Global Variables:

In [2]:
import pandas as pd
import numpy as np
import random

NUM_OF_ROWS = 25000
ATTACK_NAME = 'DoS'

In [3]:
# the following command will make it so that when we print the dataframe we will see all the columns
pd.set_option('display.max_columns', None)

---

## Load the sample dataset:

In [4]:
# import the attack sample dataset
dos_samples = pd.read_csv('dos_hulk_samples.csv')
print(f'Dataset Shape: {dos_samples.shape}')
dos_samples

Dataset Shape: (20, 26)


,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,111.356950,54,489,117.466369,13798.347794,745721,455,77.356950,20,117.466369,0,0.0,0,0.0,44.517531,0.0,745721.0,2423,6276,1481,4.684305,2057.935939,1.526422,0.000486,0.015750
1,1,110.273133,54,490,115.796032,13408.721119,734434,456,76.273133,20,115.796032,0,0.0,0,0.0,43.411985,0.0,734434.0,2517,6125,1518,6.147018,1566.450545,2.938990,0.000638,0.030000
2,1,103.581159,54,489,106.707224,11386.431598,616002,455,69.581159,20,106.707224,0,0.0,0,0.0,36.297074,0.0,616002.0,2558,5537,1172,7.872250,1124.583176,3.403323,0.000889,0.036701
3,1,98.134914,54,489,100.039017,10007.805015,579010,455,64.134914,20,100.039017,0,0.0,0,0.0,31.207133,0.0,579010.0,2046,6316,1002,7.844940,1150.805460,3.761112,0.000869,0.039894
4,1,106.005140,54,490,111.461163,12423.590853,658415,456,72.005140,20,111.461163,0,0.0,0,0.0,39.673119,0.0,658415.0,1695,6572,1338,12.302013,743.292993,10.233922,0.001346,0.107045
5,1,111.033816,54,491,116.590200,13593.274658,744917,457,77.033816,20,116.590200,0,0.0,0,0.0,44.183764,0.0,744917.0,2466,6245,1486,5.088331,1900.426688,1.499933,0.000526,0.016201
6,1,109.008152,54,489,114.543318,13120.171609,616492,455,75.008152,20,114.543318,0,0.0,0,0.0,42.333131,0.0,616492.0,2102,5178,1376,6.831848,1203.041963,3.224347,0.000831,0.036073
7,1,105.042915,54,490,110.836779,12284.791494,547954,456,71.042915,20,110.836779,0,0.0,0,0.0,38.936601,0.0,547954.0,1766,4838,1339,5.482447,1406.853508,1.897453,0.000711,0.021913
8,1,113.659936,54,490,119.414020,14259.708073,777242,456,79.659936,20,119.414020,0,0.0,0,0.0,46.785897,0.0,777242.0,2125,6926,1345,19.620577,497.284054,15.220408,0.002011,0.154469
9,1,116.414261,54,490,122.295872,14956.280327,813676,456,82.414261,20,122.295872,0,0.0,0,0.0,49.521321,0.0,813676.0,2026,7231,1260,5.859747,1684.885049,2.748188,0.000594,0.027864


### Find the columns that we need to synthesis data for:

In [5]:
columns_to_gather = dos_samples.replace(0, np.nan) #replace all 0 values with null
columns_to_gather = columns_to_gather.dropna(how = 'all', axis = 1).columns.tolist() #remove all columns where there are null values
columns_to_gather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Packet Length Std',
 'Packet Length Variance',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Fwd Packet Length Std',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'ACK Flag Count',
 'RST Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

### Find an approximate minimum and maximum values of each column:

In [6]:
# find the minimum and maximum values for each column, scale the range (reduce min by 15% and increase max by 15%), and store the results in a dictionary.
min_max_dict = {col: (dos_samples[col].min() * 0.85, dos_samples[col].max() * 1.15) for col in columns_to_gather}
min_max_dict['Number of Ports'] = (1, 1)

# print the min max dictionary
for col, (min_val, max_val) in min_max_dict.items():
    print(f'{col:<30} | Min: {min_val:.2f} | Max: {max_val:.2f}')

Number of Ports                | Min: 1.00 | Max: 1.00
Average Packet Length          | Min: 79.94 | Max: 138.22
Packet Length Min              | Min: 45.90 | Max: 62.10
Packet Length Max              | Min: 414.80 | Max: 564.65
Packet Length Std              | Min: 80.17 | Max: 145.06
Packet Length Variance         | Min: 7562.30 | Max: 18298.39
Total Length of Fwd Packet     | Min: 460918.45 | Max: 984128.60
Fwd Packet Length Max          | Min: 385.90 | Max: 525.55
Fwd Packet Length Mean         | Min: 51.04 | Max: 99.12
Fwd Packet Length Min          | Min: 17.00 | Max: 23.00
Fwd Packet Length Std          | Min: 80.17 | Max: 145.06
Fwd Segment Size Avg           | Min: 23.19 | Max: 61.33
Subflow Fwd Bytes              | Min: 0.00 | Max: 935727.40
SYN Flag Count                 | Min: 1405.05 | Max: 2941.70
ACK Flag Count                 | Min: 4057.05 | Max: 8631.90
RST Flag Count                 | Min: 797.30 | Max: 2287.35
Flow Duration                  | Min: 1.88 | Max: 31.46


### Create the base attack dataset (full of zeros):

In [7]:
# creating an empty dataframe before adding values to it
dos_dataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dos_samples.columns))), columns = dos_samples.columns)
dos_dataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Find the columns with constant zero values based on samples:

In [8]:
# adding zeros to all columns that should not have any values
zero_columns = [col for col in dos_samples.columns if col not in columns_to_gather]
for col in zero_columns:
    dos_dataset[col] = int(0)
zero_columns

['Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg']

In [9]:
dos_dataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

## Filling in values based on collected samples:

## Calculate and fill values into columns that have a certain correlation between them:

A correlation between two or more columns is common in our dataset since most features are inherently related. All of them are derived from network packet traffic.<br>
For example, as the **flow duration increases**, the **packets per second** is likely to decrease. This occurs because each flow has an upper limit on duration, after which data collection stops and a new flow begins.<br>  
Similarly, the **Inter-Arrival Time (IAT)** of packets within a flow is influenced by the flow duration. Given these dependencies, <br>
the attack dataset should generate data for these columns collectively, ensuring that their inherent correlations are maintained.

### Correlation between 'SYN Flag Count' and all the following: 'ACK Flag Count', 'RST Flag Count':

In [10]:
# finding the correlation between the 'SYN Flag Count' column to the rest of the columns in order to create new data
first_correlation = ['SYN Flag Count', 'ACK Flag Count', 'RST Flag Count']
independent_col = dos_samples[first_correlation[0]].values.reshape(-1, 1) #column 'SYN Flag Count'
dependent_cols = dos_samples[first_correlation[1:]].values 

# using least squares regression to find scaling factors that best approximate the relationship between 'SYN Flag Count' and the rest
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('ACK Flag Count', np.float64(2.8047504630690376))
('RST Flag Count', np.float64(0.6235504657577599))


After finding the scaling factors we can apply some randomness when generating values for the attack dataset in order to generate better data (without many duplications).<br>
We add randomness by creating a modified scaling factor, which introduces controlled variations in the generated values.<br>
This is done by selecting a small random delta (between 10% and 20% of the factor) and adding or subtracting it from the original scaling factor.<br>
As a result, the generated data maintains realistic correlations while avoiding exact duplicates.

In [11]:
dos_dataset['SYN Flag Count'] = np.random.randint(min_max_dict['SYN Flag Count'][0]*0.85, min_max_dict['SYN Flag Count'][1]*1.15, NUM_OF_ROWS)

# generate new data by scaling the original correlated column value using the updated factor
for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all generated scaling factors
        delta = random.uniform(factor * 0.1, factor * 0.2)
        updated_factor = factor + random.choice([-1, 1]) * delta
        dos_dataset.loc[index, col] = int(row['SYN Flag Count'] * updated_factor)

In [12]:
dos_dataset.head(10)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1362,4402.0,726.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1849,6099.0,945.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2943,9460.0,1584.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3323,10617.0,1759.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1886,6167.0,1401.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1942,4694.0,1087.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3214,10400.0,1606.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2826,6841.0,1988.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3379,8485.0,2386.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2570,6112.0,1919.0,0.0,0.0,0.0,0.0,0.0


### Correlation between 'Flow Duration' and all of the following: 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std':

In [13]:
# generate random values for the 'Flow Duration' column
rand_values = np.random.uniform(min_max_dict['Flow Duration'][0]*0.9, min_max_dict['Flow Duration'][1]*1.05, size = NUM_OF_ROWS)

# assign the random values
dos_dataset['Flow Duration'] = rand_values

In [14]:
# finding the correlation between the 'Flow Duration' column to the rest of the columns in order to create new data
second_correlation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dos_samples[second_correlation[0]].values.reshape(-1, 1) #column 'Flow Duration'
dependent_cols = dos_samples[second_correlation[1:]].values 

# using least squares regression to find scaling factors that best approximate the relationship between 'Flow Duration' and the rest of the columns in second_correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(second_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packets Per Second', np.float64(69.89957250146624))
('IAT Max', np.float64(0.6987469736662064))
('IAT Mean', np.float64(0.00010616366098115251))
('IAT Std', np.float64(0.00723950535260706))


In [15]:
# calculate the average correlation between flow duration and packets per second by multiplying their corresponding values from both columns and then calculate the average.
duration_to_packets_corr = [x * y for x, y in zip(dos_samples['Flow Duration'].values, dos_samples['Packets Per Second'].values)]
duration_to_packets_corr = np.mean(duration_to_packets_corr)
duration_to_packets_corr

np.float64(9098.6)

And again here after finding the scaling factors we add some randomness and generate the data

In [16]:
# calculate a random small delta of the factor for some randomness
for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Flow Duration'
        if col == 'Packets Per Second':
            delta = random.uniform(duration_to_packets_corr * 0.035, duration_to_packets_corr * 0.175)
            updated_factor = duration_to_packets_corr + random.choice([-1, 1]) * delta
            dos_dataset.loc[index, col] = updated_factor / row['Flow Duration']
        else:
            if col == 'IAT Std' or col == 'IAT Max':
                delta = random.uniform(factor * 0.7, factor * 0.95)
                updated_factor = factor + random.choices([-1, 1], weights=[2, 1], k=1)[0] * delta
            else:
                delta = random.uniform(factor * 0.05, factor * 0.15) 
                updated_factor = factor + random.choice([-1, 1]) * delta
            dos_dataset.loc[index, col] = row['Flow Duration'] * updated_factor

### Correlation between 'Average Packet Length' and all of the following:<br>'Packet Length Std', 'Packet Length Variance', 'Total Length of Fwd Packet', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Fwd Segment Size Avg', 'Subflow Fwd Bytes':

In [17]:
# finding the correlation between the 'Average Packet Length' column to the rest of the columns in order to create new data
third_correlation = ['Average Packet Length', 'Packet Length Std', 'Packet Length Variance', 'Total Length of Fwd Packet', 
                    'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Fwd Segment Size Avg']
independent_col = dos_samples[third_correlation[0]].values.reshape(-1, 1) #column 'Average Packet Length'
dependent_cols = dos_samples[third_correlation[1:]].values 

# using least squares regression to find scaling factors that best approximate the relationship between Average Packet Length' and the rest of the columns in third_correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(third_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packet Length Std', np.float64(1.046813231139708))
('Packet Length Variance', np.float64(119.85643269566556))
('Total Length of Fwd Packet', np.float64(6277.190505237923))
('Fwd Packet Length Mean', np.float64(0.6879594450377942))
('Fwd Packet Length Std', np.float64(1.046813231139708))
('Fwd Segment Size Avg', np.float64(0.3873521190030305))


And again here after finding the scaling factors we add some randomness and generate the data

In [18]:
dos_dataset['Average Packet Length'] = np.random.uniform(min_max_dict['Average Packet Length'][0]*0.85, min_max_dict['Average Packet Length'][1]*1.15, NUM_OF_ROWS)

for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Average Packet Length'
        delta = random.uniform(factor * 0.1, factor * 0.2) 
        updated_factor = factor + random.choice([-1, 1]) * delta
        dos_dataset.loc[index, col] = row['Average Packet Length'] * updated_factor

### Then we insert data into columns that are independant of each other:

In [19]:
independant_columns = ['Packet Length Min', 'Packet Length Max', 'Fwd Packet Length Max', 'Fwd Packet Length Min']

for col in independant_columns:
    dos_dataset[col] = (np.random.uniform(min_max_dict[col][0]*0.85, min_max_dict[col][1]*1.15, NUM_OF_ROWS)).astype(int)

dos_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,157.032114,70,441,192.240657,16051.572222,869040.073079,338,87.699995,14,184.033862,0,0,0,0,69.857577,0,0.0,1362,4402.0,726.0,23.495292,420.400739,29.559196,0.002829,0.012734
1,0.0,129.676991,56,614,149.406645,12571.228482,898988.512441,437,102.681607,16,159.456281,0,0,0,0,57.889201,0,0.0,1849,6099.0,945.0,21.151779,457.704379,26.524602,0.002039,0.040581
2,0.0,92.415522,47,359,81.518983,9342.523391,507969.236120,368,74.992729,15,114.506772,0,0,0,0,30.647620,0,0.0,2943,9460.0,1584.0,6.286134,1198.647358,0.854488,0.000586,0.004252
3,0.0,121.934575,68,413,102.901653,12833.239448,901502.651137,528,73.558500,24,110.963727,0,0,0,0,53.007476,0,0.0,3323,10617.0,1759.0,20.576528,405.592689,1.579712,0.001943,0.007571
4,0.0,111.328083,59,403,130.227707,15717.386223,585349.968956,373,67.189578,15,139.004086,0,0,0,0,48.345092,0,0.0,1886,6167.0,1401.0,21.664724,479.780147,2.831602,0.002442,0.293645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0.0,104.425023,59,490,123.255620,14996.673017,525673.324699,539,63.390679,21,95.514471,0,0,0,0,33.337853,0,0.0,1490,3419.0,1038.0,27.068323,388.580291,3.274097,0.002698,0.013565
24996,0.0,116.387115,68,566,97.628565,12490.851936,838163.784695,362,92.534845,20,101.942614,0,0,0,0,50.525745,0,0.0,2914,9548.0,2108.0,6.362214,1662.040956,8.001830,0.000742,0.012691
24997,0.0,129.322978,53,428,110.427171,17177.194530,958109.493805,407,79.663729,23,108.463730,0,0,0,0,55.277452,0,0.0,2600,6440.0,1910.0,5.659379,1399.958593,7.075821,0.000670,0.072374
24998,0.0,78.901674,45,530,66.143932,10817.304394,573463.440924,360,48.163235,24,72.686616,0,0,0,0,34.559752,0,0.0,3071,10003.0,2207.0,6.877135,1256.631990,8.942160,0.000678,0.012056


In our sample dataset, the column 'Subflow Fwd Bytes' usually has values in a specific range, but sometimes it has zero values.<br>
In order to generate accurate data, we generate a vector that will have a certain distribution of values. For example, in the 'Subflow Fwd Bytes' column, <br>
50% of the values will be within the usual range, but the other 50% will have zero values.  

In [20]:
# generate a vector with random values based on min max dict, and also create a zero vector
col = 'Subflow Fwd Bytes'
subflow_values = dos_samples[dos_samples[col] != 0][col] 
min_max_dict[col] = (np.min(subflow_values), np.max(subflow_values))

rand_values = np.random.uniform(min_max_dict[col][0]*0.9, min_max_dict[col][1]*1.1, NUM_OF_ROWS)
zero_values = np.zeros(NUM_OF_ROWS)

# choose values randomly (50% from rand_values, 50% from zero_values)
dos_dataset[col] = np.where(np.random.rand(NUM_OF_ROWS) > 0.5, rand_values, zero_values)

---

## Adding the Label column:

In [21]:
# adding number of ports and a label to the dataset
dos_dataset['Number of Ports'] = np.full(shape = NUM_OF_ROWS, fill_value = 1, dtype=int)
dos_dataset['Label'] = ATTACK_NAME

## Validate that the generated data looks valid by comparing the samples with the generated dataset:

In [22]:
dos_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std,Label
0,1,157.032114,70,441,192.240657,16051.572222,869040.073079,338,87.699995,14,184.033862,0,0,0,0,69.857577,0,0.000000,1362,4402.0,726.0,23.495292,420.400739,29.559196,0.002829,0.012734,DoS
1,1,129.676991,56,614,149.406645,12571.228482,898988.512441,437,102.681607,16,159.456281,0,0,0,0,57.889201,0,646099.234495,1849,6099.0,945.0,21.151779,457.704379,26.524602,0.002039,0.040581,DoS
2,1,92.415522,47,359,81.518983,9342.523391,507969.236120,368,74.992729,15,114.506772,0,0,0,0,30.647620,0,433303.772845,2943,9460.0,1584.0,6.286134,1198.647358,0.854488,0.000586,0.004252,DoS
3,1,121.934575,68,413,102.901653,12833.239448,901502.651137,528,73.558500,24,110.963727,0,0,0,0,53.007476,0,361478.850727,3323,10617.0,1759.0,20.576528,405.592689,1.579712,0.001943,0.007571,DoS
4,1,111.328083,59,403,130.227707,15717.386223,585349.968956,373,67.189578,15,139.004086,0,0,0,0,48.345092,0,0.000000,1886,6167.0,1401.0,21.664724,479.780147,2.831602,0.002442,0.293645,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1,104.425023,59,490,123.255620,14996.673017,525673.324699,539,63.390679,21,95.514471,0,0,0,0,33.337853,0,0.000000,1490,3419.0,1038.0,27.068323,388.580291,3.274097,0.002698,0.013565,DoS
24996,1,116.387115,68,566,97.628565,12490.851936,838163.784695,362,92.534845,20,101.942614,0,0,0,0,50.525745,0,0.000000,2914,9548.0,2108.0,6.362214,1662.040956,8.001830,0.000742,0.012691,DoS
24997,1,129.322978,53,428,110.427171,17177.194530,958109.493805,407,79.663729,23,108.463730,0,0,0,0,55.277452,0,0.000000,2600,6440.0,1910.0,5.659379,1399.958593,7.075821,0.000670,0.072374,DoS
24998,1,78.901674,45,530,66.143932,10817.304394,573463.440924,360,48.163235,24,72.686616,0,0,0,0,34.559752,0,0.000000,3071,10003.0,2207.0,6.877135,1256.631990,8.942160,0.000678,0.012056,DoS


In [23]:
dos_samples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,20.0,20.000000,20.0,20.00000,20.000000,20.000000,20.000000,20.00000,20.000000,20.0,20.000000,20.0,20.0,20.0,20.0,20.000000,20.0,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,1.0,108.647642,54.0,489.60000,113.676526,12974.395416,680262.400000,455.60000,74.647642,20.0,113.676526,0.0,0.0,0.0,0.0,41.891772,0.0,502301.225000,2140.900000,6104.800000,1339.450000,9.143862,1566.237404,5.426407,0.000991,0.056819
std,0.0,5.978754,0.0,0.88258,7.401483,1635.250105,91603.934672,0.88258,5.978754,0.0,7.401483,0.0,0.0,0.0,0.0,6.017873,0.0,284714.714226,277.269183,776.939651,281.706169,7.000491,1041.162374,6.303863,0.000718,0.064231
min,1.0,94.046800,54.0,488.00000,94.322945,8896.817984,542257.000000,454.00000,60.046800,20.0,94.322945,0.0,0.0,0.0,0.0,27.286124,0.0,0.000000,1653.000000,4773.000000,938.000000,2.208766,347.368272,0.038097,0.000222,0.000629
25%,1.0,105.764584,54.0,489.00000,110.845411,12286.705068,608590.750000,455.00000,71.764584,20.0,110.845411,0.0,0.0,0.0,0.0,39.306857,0.0,353766.125000,1969.500000,5490.500000,1127.750000,4.629564,716.920111,1.386059,0.000511,0.014721
50%,1.0,109.562540,54.0,490.00000,115.248992,13282.629414,677778.500000,456.00000,75.562540,20.0,115.248992,0.0,0.0,0.0,0.0,42.951552,0.0,601179.500000,2127.000000,6162.500000,1342.000000,6.003383,1416.331115,2.843589,0.000706,0.028932
75%,1.0,111.437199,54.0,490.00000,117.660900,13844.201033,745118.000000,456.00000,77.437199,20.0,117.660900,0.0,0.0,0.0,0.0,44.720695,0.0,727346.500000,2378.000000,6441.500000,1494.000000,12.737398,1957.185532,7.858563,0.001401,0.088955
max,1.0,120.188337,54.0,491.00000,126.141353,15911.640831,855764.000000,457.00000,86.188337,20.0,126.141353,0.0,0.0,0.0,0.0,53.333870,0.0,813676.000000,2558.000000,7506.000000,1989.000000,27.357133,4495.270243,20.157180,0.002879,0.207831


In [24]:
dos_dataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,2.500000e+04,25000.000000,25000.000000,25000.000000,25000.000000,25000.0,25000.0,25000.0,25000.0,25000.000000,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,1.0,113.421463,54.655760,500.785440,118.770516,13589.947919,7.120964e+05,464.893560,78.100775,19.970240,118.825669,0.0,0.0,0.0,0.0,43.918687,0.0,297146.160643,2291.582560,6425.749920,1429.012440,17.337657,863.263022,8.781627,0.001843,0.091183
std,0.0,26.221560,9.349252,85.375612,33.191782,3802.107773,1.989614e+05,79.820034,21.853022,3.494236,33.401435,0.0,0.0,0.0,0.0,12.306344,0.0,321659.328760,632.184634,2051.494393,455.415012,9.036903,869.035986,11.606893,0.000987,0.120479
min,1.0,67.953763,39.000000,352.000000,57.465385,6520.154337,3.427879e+05,328.000000,37.528916,14.000000,57.130567,0.0,0.0,0.0,0.0,21.209104,0.0,0.000000,1194.000000,2699.000000,598.000000,1.690653,228.605708,0.065499,0.000156,0.000626
25%,1.0,90.761971,47.000000,427.000000,92.695342,10602.362017,5.567790e+05,396.000000,60.998989,17.000000,92.957411,0.0,0.0,0.0,0.0,34.226856,0.0,0.000000,1740.750000,4764.000000,1062.000000,9.508515,360.183260,1.334091,0.000993,0.013695
50%,1.0,113.313175,55.000000,501.000000,115.824674,13253.552042,6.956006e+05,465.000000,76.155413,20.000000,115.766992,0.0,0.0,0.0,0.0,42.818679,0.0,0.000000,2296.000000,6282.000000,1396.000000,17.384314,525.852434,2.996183,0.001829,0.030751
75%,1.0,135.987110,63.000000,575.000000,140.478538,16029.209332,8.397242e+05,534.000000,92.298292,23.000000,140.527983,0.0,0.0,0.0,0.0,51.813811,0.0,593610.065686,2840.250000,7787.250000,1729.000000,25.097586,959.442986,12.007802,0.002641,0.126383
max,1.0,158.943953,71.000000,649.000000,199.523268,22814.996416,1.194849e+06,604.000000,130.941510,26.000000,199.108279,0.0,0.0,0.0,0.0,73.603049,0.0,894996.410076,3381.000000,11324.000000,2523.000000,33.032283,6266.026352,44.665979,0.004016,0.464668


---

## At the end we save the dataset as a CSV file

In [25]:
print(f'Attack Dataset Shape: {dos_dataset.shape}')

Attack Dataset Shape: (25000, 27)


In [26]:
# save the dataset
dos_dataset.to_csv('dos_hulk_dataset.csv', index=False)

---